# SU($N$), Spin $S$ antiferromagnet

This Hamiltonian has been implemented for {cite}`schwab2023phase`. It defines an SU($N$) antiferromagnetic spin-$S$ Heisenberg model, for representations given by a Young tableau with $N/ 2$ rows and $2S$ columns (c.f. {numref}`fig:young`. Specifically, the model is:

```{math}
:label: eq:hamilton-afm-n-s
  \hat{H} = J\sum_{\langle i, j  \rangle, a } \hat{S}^{(a)}_i\hat{S}^{(a)}_j,
```
where the sum extends over the pair of nearest-neighbor sites $\langle i, j \rangle$, and $a$ runs over $N^2 - 1$ generators  of the said representation of the $\mathfrak{su}(N)$ algebra of SU($N$). 

```{figure} young.svg
---
name: fig:young
---
Young tableau corresponding to the irreducible representation of $\mathfrak{su}(N)$ considered here; $N$ is even and $S$ is semi-integer.
```

The actually simulated Hamiltonian is
\begin{equation}
  \begin{split}
    \hat{H}_{\text{QMC}} =&\hat{H}_J + \hat{H}_{\text{Casimir}} + \hat{H}_U \\
    =&- \frac{J}{4}  \sum_{\langle i,j \rangle, \alpha,\beta } \left\{ \hat{D}_{(i,\alpha),(j,\beta)}, \hat{D}^{\dagger}_{(i,\alpha),(j,\beta)}   \right\} \\
    &+ \frac{J_H}{2} \sum_{i} \sum_{\alpha>\beta}    \left\{ \hat{D}_{(i,\alpha),(i,\beta)}, \hat{D}^{\dagger}_{(i,\alpha),(i,\beta)}   \right\}\\
    &+ U\sum_{i,\alpha}\left(\hat{n}_{i,\alpha}-\frac{N}{2}\right)^2,\\
  \end{split}
\end{equation}
where
\begin{equation}
  \hat{D}_{(i,\alpha),(j,\beta)} \equiv  \sum_{\sigma} \hat{c}_{i,\alpha,\sigma}^{\dagger}\hat{c}_{j,\beta,\sigma}^{\phantom{\dagger}},
  \qquad
  \left\{\hat{A}, \hat{B}\right\} \equiv \hat{A} \hat{B} + \hat{B} \hat{A},
  \qquad
  \hat{n}_{i,\alpha} \equiv \sum_{\sigma=1}^{N} \hat{c}_{i,\alpha,\sigma}^{\dagger}\hat{c}_{i,\alpha,\sigma}^{\phantom{\dagger}}.
\end{equation}

The terms with the parameters $J_{\text{H}}$ and $U$ are projecting unto the desired Hilbert space to achieve Eq.&nbsp;{eq}`eq:hamilton-afm-n-s`.

The following square lattice geometries are implemented:
- periodic boundary conditions (`Lattice_type = Square`)
- open boundary conditions in one dimension and periodic boundary conditions in the other (`Lattice_type = N_leg_ladder`)
- open boundary conditions (`Lattice_type = Open_square`)

Extending to other lattices should be relatively easy. If the lattices are bipartite, the simulations will be sign-problem free.

The following model-specific parameters are available:

```fortan
Character (len=64) :: Lattice_type = 'Square'  ! Implemented: Square, N_leg_ladder and Open_square
Integer :: L1 = 4                  ! Size of lattice in first dimension
Integer :: L2 = 4                  ! Size of lattice in second dimension
!Integer :: N_SUN = 2              ! SU(N) symmtry
Integer :: N_Spin = 1              ! 2*S
!logical :: Projector = .false.    ! Use projetive finite temperature method
!logical :: Symm = .false.         ! Use symmetric Trotter decomposition
real(dp) :: ham_J = 1.d0           ! Antiferromagnetic interaction strength
real(dp) :: ham_U = 1.d0           ! On-site Hubbard term for freezing out charge fluctuations
real(dp) :: ham_J2 = -1.d0         ! Projection parameter for going to fully symmetric representation
real(dp) :: dtau = 0.1d0           ! Imaginary time step
real(dp) :: beta = 5.d0            ! Reciprcal temperature
real(dp) :: theta = 0.d0           ! Zero-temperature projection parameter
real(dp) :: pinning_factor = 1.d0  ! Factor by which a single J-value will be multiplied to achieve phase pinning.
integer :: pinning_x = 0           ! x coordinate of pinned bond
integer :: pinning_y = 0           ! y coordinate of pinned bond
```

## Usage example

A brief usage example using [pyALF](https://git.physik.uni-wuerzburg.de/ALF/pyALF).

### Use provided script to download ALF and link the Hamiltonian to it

In [1]:
!./clone_alf.sh

Cloning into 'ALF'...
remote: Enumerating objects: 29841, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 29841 (delta 185), reused 0 (delta 0), pack-reused 29545 (from 1)
Receiving objects: 100% (29841/29841), 243.82 MiB | 92.40 MiB/s, done.
Resolving deltas: 100% (21204/21204), done.


### Import necessary pyALF classes.

In [2]:
from py_alf import ALF_source, Simulation

### Create ALF source object and inspect available parameters

In [3]:
alf_src = ALF_source(alf_dir='./ALF')

In [4]:
alf_src.get_ham_names()

['Kondo',
 'Hubbard',
 'Hubbard_Plain_Vanilla',
 'tV',
 'LRC',
 'Z2_Matter',
 'Spin_Peierls',
 'AFM_N_S']

In [5]:
alf_src.get_default_params('AFM_N_S', include_generic=False)

OrderedDict([('VAR_AFM_N_S',
              {'Lattice_type': {'comment': 'Implemented: Square, N_leg_ladder and Open_square',
                'defined_in_base': False,
                'value': 'Square'},
               'L1': {'comment': 'Size of lattice in first dimension',
                'defined_in_base': False,
                'value': 4},
               'L2': {'comment': 'Size of lattice in second dimension',
                'defined_in_base': False,
                'value': 4},
               'N_SUN': {'comment': 'SU(N) symmtry',
                'defined_in_base': True,
                'value': 2},
               'N_Spin': {'comment': '2*S',
                'defined_in_base': False,
                'value': 1},
               'Projector': {'comment': 'Use projetive finite temperature method',
                'defined_in_base': True,
                'value': False},
               'Symm': {'comment': 'Use symmetric Trotter decomposition',
                'defined_in_base': True,
  

### Create and run Simulation

In [6]:
sim = Simulation(
    alf_src, "AFM_N_S",
    {
        'L1': 4,
        'L2': 4,
        'Projector': True,
        'Symm': True,
        'N_Spin': 2,
        'theta': 2.,
        'beta': 0.5,
     },
     machine='GNU',
     mpi=True,
    )

In [7]:
# On first compile, an ALF-local version of HDF5 will be compiled, 
# which takes several minutes and echos many warnings.
sim.compile()

========== Downloading HDF5 source ==========
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.8M  100 11.8M    0     0  24.3M      0 --:--:-- --:--:-- --:--:-- 24.3M
=== Build with the following compilers C: gcc, Fortran: gfortran, C++: g++ 
In file included from ../hdf5-1.10.7/src/H5.c:22:
../hdf5-1.10.7/src/H5.c: In function ‘H5check_version’:
../hdf5-1.10.7/src/H5private.h:1470:31: warning: ‘strncat’ specified bound 1 equals source length [-Wstringop-overflow=]
 1470 |     #define HDstrncat(X,Y,Z)  strncat(X,Y,Z)
      |                               ^~~~~~~~~~~~~~
../hdf5-1.10.7/src/H5.c:889:13: note: in expansion of macro ‘HDstrncat’
  889 |             HDstrncat(lib_str, "-", (size_t)1);
      |             ^~~~~~~~~
In file included from ../hdf5-1.10.7/src/H5system.c:33:
../hdf5-1.10.7/src/H5system.c: In function ‘H5_build_extpath’:
../hdf5-1.10.7/src/H5private.h:

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a
ar: creating libqrref.a


Compiling Analysis
Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_AFM_N_S_smod.F90 Hamiltonians/Hamiltonian_AFM_N_S_read_write_parameters.F90
Compiling progra

In [8]:
sim.run()

Prepare directory "/home/jonas/Programs/hamiltonians/Hamiltonians/AFM_N_S/ALF_data/AFM_N_S_L1=4_L2=4_Projector=True_Symm=True_N_Spin=2_theta=2.0_beta=0.5" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/hamiltonians/Hamiltonians/AFM_N_S/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 gap:    4.9347005129783043E-004   4.9347005129783043E-004
 No initial configuration
